In [1]:
import pathlib
import pandas as pd
import geopandas as gpd
import rasterio
from zipfile import ZipFile
import fnmatch
import glob

In [2]:
# Read the envelopes GeoDataFrame
envelopes_gdf = gpd.read_file(r"C:\Users\Kostas\Desktop\GIMA\Module_7\Data\PEP725\After_2016_sent_from_PEP725\pep725_outputs\PEP725_envelopes.geojson")
envelopes_gdf.set_crs(32632, inplace=True, allow_override=True)
envelopes_list = envelopes_gdf.geometry.tolist()

In [3]:
envelopes_gdf.head()

,s_id,lon,lat,alt,alt_dem,gss_id,genus,species,phase_id,year,day,date,Label,geometry
0,5363,13.91670,54.0833,2,0,1050100,Alnus,Alnus glutinosa,60,2017,27,2017-01-27,DBL,"POLYGON ((817520.468 5999973.720, 825520.468 5..."
1,1554,7.51667,51.7333,60,72,2210500,Salix,Salix caprea,60,2017,29,2017-01-29,DBL,"POLYGON ((393567.206 5728416.903, 401567.206 5..."
2,3120,8.68333,49.5500,140,261,1050100,Alnus,Alnus glutinosa,60,2017,31,2017-01-31,DBL,"POLYGON ((473094.080 5484647.767, 481094.080 5..."
3,2021,8.58333,50.0000,100,101,1050100,Alnus,Alnus glutinosa,60,2017,32,2017-02-01,DBL,"POLYGON ((466138.525 5534713.881, 474138.525 5..."
4,1521,7.83333,51.7000,60,58,1050100,Alnus,Alnus glutinosa,60,2017,33,2017-02-02,DBL,"POLYGON ((415374.473 5724316.443, 423374.473 5..."


Temporal filter

In [4]:
import glob
from zipfile import ZipFile
import fnmatch
import datetime
from datetime import timedelta

sentinel_2_directory = r"C:\Users\Kostas\Desktop\GIMA\Module_7\Data\Sentinel2_images"

# Create a list of all the Sentinel-2 zipfiles
sentinel_2_zip_list = glob.glob(str(sentinel_2_directory) + '/*.zip', recursive=True)

# Get the date from the Sentinel-2 zip archive name. Returns a datetime class object
def getS2Date(s2zip_path):
    string_parts = s2zip_path.split("_")
    band_string = string_parts[-1]
    band_string = band_string.replace('.zip','')
    string_parts = band_string.split("T")
    s2_date = string_parts[0]
    s2_time = string_parts[1]
    s2_date = datetime.datetime.strptime(s2_date, "%Y%m%d").date()
    #print(band_string)
    #print(s2_date, s2_time)
    return s2_date

In [5]:
# Extract a list with the stations.
stations_list = envelopes_gdf['s_id'].unique().tolist()

# Converting the date column to datetime data type
envelopes_gdf['date'] = pd.to_datetime(envelopes_gdf['date'], format='%Y-%m-%d').dt.date

In [6]:
# New try 22feb
for i_id in range(0, len(sentinel_2_zip_list) - 1):
    print(i_id)
    print(sentinel_2_zip_list[i_id])
    image_1 = sentinel_2_zip_list[i_id]
    image_2 = sentinel_2_zip_list[i_id + 1]

    date_start = getS2Date(image_1)
    date_end = getS2Date(image_2)
    print(str(date_start), str(date_end))
    # create an empty list to store the unique station results
    results = []
    freqresults = []

    # Create an empty dataframe to store the frequency results
    freqresults_df = pd.DataFrame(columns=['s_id', 'max_label', 'max_phase_id'])

    # iterate over the station IDs
    for s_id in stations_list:
        # Filter the geodataframe to include only the rows with the current station ID
        # mask is a boolean Series, with True in the places where the station IDs match.
        mask = envelopes_gdf['s_id'] == s_id
        
        # Filter the geodataframe to include only the rows with dates between 'date_start' and 'date_end'
        # Now, the dates between date_start and date_end are assigned a True value
        # Because &= is used, the ultimate values that kept are the ones that meet both the station matching and the date matching criteria
        # The mask is then updated to be used next
        mask &= (envelopes_gdf['date'] >= date_start) & (envelopes_gdf['date'] < date_end)
        
        # Extract the relevant columns for the filtered rows. Here, the mask is used as an index because it shares the same indices with envelopes_gdf
        # This way the only data that are passed are the ones for which the criteria mentioned above match.
        filtered_gdf = envelopes_gdf.loc[mask, ['s_id', 'date', 'Label', 'phase_id', 'lat', 'lon']]
        
        # Append the filtered data to the results list. This will be used to concatenate them to the final_df
        results.append(filtered_gdf)
        
        # This part finds the 'Label' and 'phase_id' that has the highest frequency

        # Check if the filtered geodataframe is not empty
        if not filtered_gdf.empty:
            # Group the filtered geodataframe by station ID and label, and count the frequency of each label
            label_counts = filtered_gdf.groupby(['s_id', 'Label']).size().reset_index(name='count')
            
            # Find the label that has the highest frequency for the current station ID
            max_label = label_counts.loc[label_counts['s_id'] == s_id, 'Label'][label_counts.loc[label_counts['s_id'] == s_id, 'count'].idxmax()]

            phase_id_counts = filtered_gdf.groupby(['s_id', 'phase_id']).size().reset_index(name='count2')

            max_phase_id = phase_id_counts.loc[phase_id_counts['s_id'] == s_id, 'phase_id'][phase_id_counts.loc[phase_id_counts['s_id'] == s_id, 'count2'].idxmax()]
            
            # Add the result to the freqresults dataframe
            freqresults_df = freqresults_df.append({'s_id': s_id, 'max_label': max_label, 'max_phase_id': max_phase_id}, ignore_index=True)

    # Concatenate the filtered data from all stations into a single geodataframe
    final_df = pd.concat(results)
    break

0
C:\Users\Kostas\Desktop\GIMA\Module_7\Data\Sentinel2_images\1.S2A_MSIL2A_20200921T103031_N0214_R108_T32UNB_20200921T151006.zip
2020-09-21 2022-10-22


C:\Users\Kostas\AppData\Local\Temp\ipykernel_12956\1119309902.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  freqresults_df = freqresults_df.append({'s_id': s_id, 'max_label': max_label, 'max_phase_id': max_phase_id}, ignore_index=True)
C:\Users\Kostas\AppData\Local\Temp\ipykernel_12956\1119309902.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  freqresults_df = freqresults_df.append({'s_id': s_id, 'max_label': max_label, 'max_phase_id': max_phase_id}, ignore_index=True)
C:\Users\Kostas\AppData\Local\Temp\ipykernel_12956\1119309902.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  freqresults_df = freqresults_df.append({'s_id': s_id, 'max_label': max_label, 'max_phase_id': max_phase_id}, ignore_index=True)
C:\U

In [11]:
final_df.sort_values(['date'])

,s_id,date,Label,phase_id,lat,lon
86877,3586,2020-09-21,DBL,286,47.7667,11.73330
86894,4736,2020-09-21,DBL,286,52.1667,14.25000
86842,511,2020-09-21,DBL,286,51.5500,10.06670
86840,303,2020-09-21,DBL,286,53.8667,10.33330
86863,2485,2020-09-21,DBL,205,50.8167,7.75000
...,...,...,...,...,...,...
113480,5999,2021-12-04,DBL,95,51.7667,10.80000
113479,1296,2021-12-04,DBL,95,51.8333,6.31667
113481,1042,2021-12-05,DBL,95,53.5333,7.70000
113482,3926,2021-12-05,DBL,95,49.0000,11.91670


In [8]:
freqresults_df.head()

,s_id,max_label,max_phase_id
0,5363,DBL,60
1,3120,DBL,60
2,2021,DBL,60
3,1521,DBL,205
4,1710,DBL,60


In [27]:
unique_dates_list = final_df['date'].unique().tolist()

In [28]:
print(f"There are {len(unique_dates_list)} unique dates in the list")


There are 393 unique dates in the list


In [29]:
# Keeping the envelope_gdf of the dates in unique_dates_list

# Create a boolean mask for selecting records with dates in unique_dates_list
mask = envelopes_gdf['date'].isin(unique_dates_list)

# Filter the envelopes_gdf using the mask
envelopes_gdf_filtered_dates = envelopes_gdf[mask]

In [32]:
envelopes_gdf_filtered_dates.head()

,s_id,lon,lat,alt,alt_dem,gss_id,genus,species,phase_id,year,day,date,Label,geometry
86837,148,10.83330,54.2500,55,35,1030100,Aesculus,Aesculus hippocastanum,205,2020,265,2020-09-21,DBL,"POLYGON ((615444.341 6008888.454, 623444.341 6..."
86838,164,9.78333,53.7333,10,23,1180100,Betula,Betula pendula,205,2020,265,2020-09-21,DBL,"POLYGON ((547674.082 5950133.967, 555674.082 5..."
86839,274,10.21670,53.9333,45,60,1030100,Aesculus,Aesculus hippocastanum,286,2020,265,2020-09-21,DBL,"POLYGON ((575879.730 5972786.252, 583879.730 5..."
86840,303,10.33330,53.8667,25,38,1030100,Aesculus,Aesculus hippocastanum,286,2020,265,2020-09-21,DBL,"POLYGON ((583673.774 5965514.754, 591673.774 5..."
86841,386,9.18333,52.8500,15,16,2120200,Quercus,Quercus robur,286,2020,265,2020-09-21,DBL,"POLYGON ((508345.936 5851600.219, 516345.936 5..."


In [33]:
envelopes_gdf_filtered_dates.shape

(29736, 14)

In order to work with the spatial filter the geometries of the time-filtered envelopes is needed.

In [13]:
envelopes_gdf_filtered_dates_list = envelopes_gdf_filtered_dates['geometry'].tolist()

## Spatial filter

In [34]:
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
from shapely.geometry import box
import rasterio
from rasterio.plot import show
from rasterio.mask import mask
from rasterio.coords import BoundingBox
from rasterio.features import geometry_mask
import json
import numpy as np
import glob
import os
from rasterio.windows import from_bounds
import pathlib

Defining the functions to be used

In [35]:
"""Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""

def getFeatures(gdf):
        return [json.loads(gdf.to_json())['features'][0]['geometry']]

'''
This function reads the envelope list and a raster, checks if the polygons are fully contained in the raster 
and returns 4 lists with the boundary coordinates for all the envelopes that are fully contained in the raster.
'''

def getContainedEnvelopeCoords (raster, envelope_list):
    with rasterio.open(raster, driver='JP2OpenJPEG') as src:
        raster_extent = src.bounds
        
        # List initialization
        minx_list = []
        miny_list = []
        maxx_list = []
        maxy_list = []
        for poly in envelope_list:
            poly_extent = poly.bounds

            # Check if the polygon is fully inside the raster's extent
            if (poly_extent[0] >= raster_extent[0] and poly_extent[2] <= raster_extent[2] and
                poly_extent[1] >= raster_extent[1] and poly_extent[3] <= raster_extent[3]):
                    minx_list.append(poly_extent[0])
                    miny_list.append(poly_extent[1])
                    maxx_list.append(poly_extent[2])
                    maxy_list.append(poly_extent[3])
    return minx_list, miny_list, maxx_list, maxy_list
'''
This function receives a raster file (.jp2) and the boundary coordinates for a polygon. 
It then clips the raster to the extent of the polygon. 
The polygon has to intersect the raster for the operation to be completed
'''

def exportImage(raster, output_path, minx, miny, maxx, maxy):
    # open the raster file (Single Band)
    data = rasterio.open(raster, driver='JP2OpenJPEG')

    # Create a bounding box from the polygon min-max coordinates    
    bbox = box(minx, miny, maxx, maxy)
    # Create a geodataframe with a single polygon so that it can be used with rasterio
    geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs='32632')
    # Transform the geodataframe to a GeoJSON-like object that can be used as an input in the rasterio mask function
    coords = getFeatures(geo)
    #print(coords)
    
    # Mask and crop the raster AOI where polygon overlaps the whole raster
    out_img, out_transform = mask(data, shapes=coords, crop=True)
    # Define resolution and more
    out_profile = data.profile.copy()
    
    out_profile.update({'driver':'PNG', 'width': out_img.shape[2],'height': out_img.shape[1], 'transform': out_transform})
    
    # Write the extracted raster patch to a file
    with rasterio.open(output_path, 'w', **out_profile) as dst:
        dst.write(out_img)
    
    # data.close()
    # data = None
'''
This function is used to receive a string from the raster file's name
using the split() method. It splits the string wherever an underscore appears and then accesses the second-to-last element.
'''
def imageNaming(raster_path):
    string_parts = raster_path.split("_")
    band_string = string_parts[-3]+ "_" + string_parts[-2] + "_" + string_parts[-1]
    band_string = band_string.replace('.jp2','')
    return band_string

# Example output: 20170420T103021_B02_10m
'''
This function is used to mask an already created patch based on CLC. It reads the CLC raster, the list of the CLC pixel values that the mask will
have and the output path, which should be the same if you don't want to keep the original patch.
It opens the patch and gets its boundary
Then it opens the CLC raster and limits the operation based on the patch boundary
Then, it creates a mask on the CLC raster based on the clc_pixel_values list
Because of the way the mask is created it creates nodata values. Rasterio needs the nodata values to be numbers, so the next step replaces the string "nodata" with an integer (0)
Finally, it exports the masked patch
'''

def pixelMasking(patch_path, clc_path, clc_pixel_values, output_path):
    # Read the patch and its boundaries
    with rasterio.open(patch_path) as patch:
        patchdata = patch.read()
        profile = patch.profile.copy()
        bounds = patch.bounds

    # Read clc in the boundaries of the patch
    with rasterio.open(clc_path) as clc:
        window = from_bounds(*bounds, clc.transform)
        clcdata = clc.read(window=window)

    # Create bool array mask where clc is not in list of values
    mask = np.isin(clcdata, clc_pixel_values, invert=True)

    # Replace "None" values with integer. In this case 0
    if profile["nodata"] is None:
        profile["nodata"] = 0
        
    # set everything where mask is True to NoData
    patchdata[mask] = profile["nodata"]

    # Save the new patch with the patch values where clc has specific values
    with rasterio.open(output_path, "w", **profile) as output:
        output.write(patchdata)
   

Using the spatial filter

In [36]:
from zipfile import ZipFile
import fnmatch
import os

# Make a variable from the date-filtered envelope list. It is used in the getContainedEnvelopeCoords function
envelope_list = envelopes_gdf_filtered_dates_list


# Read clc related info
clc_pixel_values = [10, 23, 25, 29, 24, 18, 26]
clc_path = pathlib.Path(r"C:\Users\Kostas\Desktop\GIMA\Module_7\Data\CLC2018\CLC2018_GER_WGS84UTM32N_60m_ArcPro.tif")

# Read the output directory for the images that will be extracted
#output_path = pathlib.Path(r'C:\Users\Kostas\Desktop\GIMA\Module_7\Data\filtered_patches')
output_path = pathlib.Path(r"C:\Users\Kostas\Desktop\GIMA\Module_7\Data\Sentinel2_images\test")

# Create a list of the existing Sentinel-2 zip files
sentinel_2_directory = pathlib.Path(r'C:\Users\Kostas\Desktop\GIMA\Module_7\Data\Sentinel2_images')
sentinel_2_zip_list = glob.glob(str(sentinel_2_directory) + '/*.zip', recursive=True)

# Create a list of the band paths of Sentinel-2 zips
for i_id in range(0, len(sentinel_2_zip_list) - 1):
    zipaki = sentinel_2_zip_list[i_id]
    print('---------------')

    # This part is a filter to find only the relevant 60m bands from inside the Sentinel-2 zip. Maybe I should wrap this up into a function as well.
    with ZipFile(zipaki, 'r') as zipObj:
        file_list = zipObj.namelist()
        pattern = '*/R60m/*B???60m.jp2'

        filtered_list = []
        for file in file_list:
            if fnmatch.fnmatch(file, pattern):
                filtered_list.append(file)
        print(f'List of spectral bands for file {zipaki} completed.')

        # Now read each band (each band is a different raster)
        for raster in filtered_list:
            zipped_image = pathlib.Path("zip+file:" + zipaki + '!/' + raster)
            # Spatial filter. Has all the envelope coordinates that are within a Sentinel-2 image
            minx_list, miny_list, maxx_list, maxy_list = getContainedEnvelopeCoords(zipped_image, envelope_list)
            raster_name = imageNaming(raster)
            print("\tCreating images around PEP725 stations for the band:", raster_name)

            # For each envelope in the band, export the image and use the clc related info to mask the pixels
            for j in range(0, len(minx_list)):
                # Temporal filter should probably go here
                output_file_name = os.path.join(output_path, raster_name + f'_Patch_{j+1}.png')
                exportImage(zipped_image, output_file_name, minx_list[j], miny_list[j], maxx_list[j], maxy_list[j])
                pixelMasking(output_file_name, clc_path, clc_pixel_values, output_file_name)
                break
            print(f'\tPatch pixel masking completed for all patches of the band {raster_name}\n')
        print(f'\tImage extraction completed for the file{zipaki}')
print('Image extraction completed for all the files')


---------------
List of spectral bands for file C:\Users\Kostas\Desktop\GIMA\Module_7\Data\Sentinel2_images\1.S2A_MSIL2A_20200921T103031_N0214_R108_T32UNB_20200921T151006.zip completed.
	Creating images around PEP725 stations for the band: 20200921T103031_B01_60m
	Patch pixel masking completed for all patches of the band 20200921T103031_B01_60m

	Creating images around PEP725 stations for the band: 20200921T103031_B02_60m
	Patch pixel masking completed for all patches of the band 20200921T103031_B02_60m

	Creating images around PEP725 stations for the band: 20200921T103031_B03_60m
	Patch pixel masking completed for all patches of the band 20200921T103031_B03_60m

	Creating images around PEP725 stations for the band: 20200921T103031_B04_60m
	Patch pixel masking completed for all patches of the band 20200921T103031_B04_60m

	Creating images around PEP725 stations for the band: 20200921T103031_B05_60m
	Patch pixel masking completed for all patches of the band 20200921T103031_B05_60m

	Cre